# 3M dataset - Data Preprocessing

 cleansing 3M flight records for delay prediction modeling.

## Contents
1. [Data Loading](#loading)
2. [Handling cancelled flihgts](#cancelled)
3. [Save Processed Dataset](#save)
4. [Handling Missing Values](#missing_values)

## 1. Data Loading <a id='loading'></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')
sns.set(style="whitegrid")

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(PROJECT_ROOT)
from src.data import loader, processor
from src.visualization import exploratory_viz

In [9]:
raw_file_path = os.path.join(PROJECT_ROOT, 'data', 'raw', 'flights_sample_3m.csv')
processed_dir = os.path.join(PROJECT_ROOT, 'data', 'processed')
os.makedirs(processed_dir, exist_ok=True)

print(f"Input data file: {raw_file_path}")
print(f"Processed data directory: {processed_dir}")

Input data file: c:\Users\HP\Desktop\Forecasting_Flights-DataScience\data\raw\flights_sample_3m.csv
Processed data directory: c:\Users\HP\Desktop\Forecasting_Flights-DataScience\data\processed


In [10]:
# Function to process data in chunks to handle 3M rows efficiently
def process_data_in_chunks(file_path, chunksize=100000):
    """
    Generator function to process the data in chunks
    Args:
        file_path: Path to the CSV file
        chunksize: Number of rows to process at a time
    Yields:
        Processed DataFrame chunks
    """
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        yield chunk

first_chunk = next(process_data_in_chunks(raw_file_path, chunksize=5))
first_chunk

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN


## 2. Handling Cancelled Flights <a id='cancelled'></a>

As we're focusing on predicting delays for flights that actually operated, we'll separate cancelled flights from our analysis dataset. Cancelled flights represent a different phenomenon and would confuse our delay prediction models.

In [11]:
# First, let's analyze how many flights were cancelled
def analyze_cancellations(file_path, chunksize=100000):
    cancelled_count = 0
    total_count = 0
    cancellation_codes = {}
    
    # Process in chunks to handle the large dataset
    for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunksize)):
        total_count += len(chunk)
        
        # Count cancelled flights
        cancelled_in_chunk = chunk[chunk['CANCELLED'] == 1.0]
        cancelled_count += len(cancelled_in_chunk)
        
        # Track cancellation reasons
        code_counts = cancelled_in_chunk['CANCELLATION_CODE'].value_counts()
        for code, count in code_counts.items():
            if code in cancellation_codes:
                cancellation_codes[code] += count
            else:
                cancellation_codes[code] = count
                
        # Print progress
        if (i+1) % 10 == 0:
            print(f"Processed {(i+1)*chunksize:,} rows...")
    
    # Calculate percentage
    cancelled_pct = (cancelled_count / total_count) * 100 if total_count > 0 else 0
    
    # Create a summary dictionary
    summary = {
        'total_flights': total_count,
        'cancelled_flights': cancelled_count,
        'cancelled_percentage': cancelled_pct,
        'cancellation_codes': cancellation_codes
    }
    
    return summary

# Run the analysis
cancellation_summary = analyze_cancellations(raw_file_path)

# Display the results
print(f"Total Flights: {cancellation_summary['total_flights']:,}")
print(f"Cancelled Flights: {cancellation_summary['cancelled_flights']:,} ({cancellation_summary['cancelled_percentage']:.2f}%)")
print("\nCancellation Codes:")
for code, count in cancellation_summary['cancellation_codes'].items():
    print(f"  {code}: {count:,} flights")
    
# Explanation of cancellation codes
code_meanings = {
    'A': 'Carrier',
    'B': 'Weather',
    'C': 'National Air System',
    'D': 'Security'
}

print("\nCancellation Code Meanings:")
for code, meaning in code_meanings.items():
    print(f"  {code}: {meaning}")

Processed 1,000,000 rows...
Processed 2,000,000 rows...
Processed 3,000,000 rows...
Total Flights: 3,000,000
Cancelled Flights: 79,140 (2.64%)

Cancellation Codes:
  B: 28,772 flights
  D: 24,417 flights
  A: 19,476 flights
  C: 6,475 flights

Cancellation Code Meanings:
  A: Carrier
  B: Weather
  C: National Air System
  D: Security


In [12]:
# Function to filter out cancelled flights and save operational flights
def filter_operational_flights(input_path, output_path, chunksize=100000):
    """
    Filter the dataset to include only operational flights (not cancelled)
    and save it to a new CSV file.
    
    Args:
        input_path: Path to the input CSV file
        output_path: Path to save the filtered CSV file
        chunksize: Number of rows to process at a time
    """
    # Track statistics
    total_rows = 0
    operational_rows = 0
    
    # Process the first chunk to get the header
    first_chunk = True
    
    # Process in chunks
    for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):
        # Update counts
        total_rows += len(chunk)
        
        # Filter out cancelled flights
        operational_chunk = chunk[chunk['CANCELLED'] != 1.0]
        operational_rows += len(operational_chunk)
        
        # Save the chunk (append mode after first chunk)
        if first_chunk:
            operational_chunk.to_csv(output_path, index=False)
            first_chunk = False
        else:
            operational_chunk.to_csv(output_path, mode='a', header=False, index=False)
        
        # Print progress
        if (i+1) % 5 == 0:
            print(f"Processed {(i+1)*chunksize:,} rows...")
    
    # Return statistics
    return {
        'total_rows': total_rows,
        'operational_rows': operational_rows,
        'filtered_rows': total_rows - operational_rows,
        'filtered_percentage': ((total_rows - operational_rows) / total_rows) * 100 if total_rows > 0 else 0
    }

operational_flights_path = os.path.join(processed_dir, 'operational_flights.csv')

# Run the filtering process
print("Filtering out cancelled flights...")
filter_stats = filter_operational_flights(raw_file_path, operational_flights_path)

# Display the results
print("\nFiltering complete!")
print(f"Total rows processed: {filter_stats['total_rows']:,}")
print(f"Operational flights: {filter_stats['operational_rows']:,}")
print(f"Cancelled flights removed: {filter_stats['filtered_rows']:,} ({filter_stats['filtered_percentage']:.2f}%)")
print(f"\nOperational flights saved to: {operational_flights_path}")

Filtering out cancelled flights...
Processed 500,000 rows...
Processed 1,000,000 rows...
Processed 1,500,000 rows...
Processed 2,000,000 rows...
Processed 2,500,000 rows...
Processed 3,000,000 rows...

Filtering complete!
Total rows processed: 3,000,000
Operational flights: 2,920,860
Cancelled flights removed: 79,140 (2.64%)

Operational flights saved to: c:\Users\HP\Desktop\Forecasting_Flights-DataScience\data\processed\operational_flights.csv


## 3. Handling Missing Values <a id='missing_values'></a>

In [13]:
import pandas as pd
import os

def handle_missing_values_in_chunks(input_path, output_path, chunksize=100000, high_thresh=0.8):
    """
    Cleans a large CSV file in chunks by:
    - Dropping columns with missing values above a threshold (based on the first chunk).
    - Filling or dropping remaining missing values.
    
    Args:
        input_path: Path to the input CSV file
        output_path: Path to save the cleaned CSV file
        chunksize: Number of rows to process at a time
        high_thresh: Threshold for high missing values (default 0.8)
    """
    first_chunk = True
    total_rows = 0
    total_kept_rows = 0
    columns_to_drop = []

    for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):
        total_rows += len(chunk)

        if first_chunk:
            # Determine columns with high missing ratio
            missing_ratio = chunk.isnull().mean()
            columns_to_drop = missing_ratio[missing_ratio > high_thresh].index.tolist()
            print(f"Dropping columns with >{int(high_thresh*100)}% missing values: {columns_to_drop}")
            first_chunk = False

        # Drop columns with high missing values
        chunk = chunk.drop(columns=columns_to_drop, errors='ignore')

        # Fill delay reason columns if they still exist
        delay_cols = ['DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY',
                      'DELAY_DUE_CARRIER', 'DELAY_DUE_LATE_AIRCRAFT']
        for col in delay_cols:
            if col in chunk.columns:
                chunk[col] = chunk[col].fillna(0)

        # Drop rows with missing values in essential timing fields
        critical_cols = ['DEP_TIME', 'ARR_TIME', 'AIR_TIME']
        existing_critical_cols = [col for col in critical_cols if col in chunk.columns]
        chunk = chunk.dropna(subset=existing_critical_cols)

        total_kept_rows += len(chunk)

        # Write chunk to file
        if i == 0:
            chunk.to_csv(output_path, index=False)
        else:
            chunk.to_csv(output_path, mode='a', header=False, index=False)

        if (i + 1) % 5 == 0:
            print(f"Processed {(i + 1) * chunksize:,} rows...")

    return {
        'total_rows': total_rows,
        'kept_rows': total_kept_rows,
        'dropped_rows': total_rows - total_kept_rows,
        'dropped_percentage': ((total_rows - total_kept_rows) / total_rows) * 100 if total_rows > 0 else 0,
        'dropped_columns': columns_to_drop
    }


In [14]:
cleaned_file_path = os.path.join(processed_dir, 'cleaned_operational_flights.csv')

print("Handling missing values in chunks...")
clean_stats = handle_missing_values_in_chunks(operational_flights_path, cleaned_file_path, chunksize=100000)

print("\nCleaning complete!")
print(f"Total rows processed: {clean_stats['total_rows']:,}")
print(f"Rows kept: {clean_stats['kept_rows']:,}")
print(f"Rows dropped: {clean_stats['dropped_rows']:,} ({clean_stats['dropped_percentage']:.2f}%)")
print(f"Dropped columns: {clean_stats['dropped_columns']}")


Handling missing values in chunks...
Dropping columns with >80% missing values: ['CANCELLATION_CODE', 'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT']
Processed 500,000 rows...
Processed 1,000,000 rows...
Processed 1,500,000 rows...
Processed 2,000,000 rows...
Processed 2,500,000 rows...


KeyboardInterrupt: 